In [43]:
import pandas as pd
import sys

sys.path.append("/var/DSTeam/locnt2/personal/quotes-tagging/")
from collections import Counter
# import torch
# from torch import nn
import numpy as np
import json
import itertools
import shutil


In [44]:
df=pd.read_csv('/var/DSTeam/locnt2/personal/quotes-tagging/quotes_clean.csv')

In [45]:
cate_roots= json.load(open('/var/DSTeam/locnt2/personal/quotes-tagging/category_raw.json'))

In [46]:
cate_pd_total=pd.DataFrame()
for big_cat in cate_roots.keys():
    print(big_cat)
    bigcate=[big_cat]
    cates=[cate_roots[big_cat][i] for i in range(len(cate_roots[big_cat]))]
    pairs=list(itertools.product(bigcate,cates))
    cate_pd=pd.DataFrame(pairs,columns=['big_cate','tag'])
    
    cate_pd_total=pd.concat([cate_pd_total,cate_pd])

hard times
personal growth
work & productivity
inspiration
relationships
spiritual & philosophy
health & fitness


In [47]:
cate_pd_total.loc[lambda x:x.big_cate=='relationships']

,big_cate,tag
0,relationships,unconditional love
1,relationships,cheating
2,relationships,marriage
3,relationships,friendship
4,relationships,parenthood
5,relationships,being single
6,relationships,loyalty
7,relationships,distance
8,relationships,falling in love
9,relationships,love


In [48]:
fake_id=list(range(len(df)))

In [49]:
df['id']=fake_id

In [50]:
df_quote_1=pd.merge(
    df.assign(tag=lambda x:x.tags.str.lower()).assign(tag=lambda x:x.tag.str.split(',')).explode('tag'),
    cate_pd_total,
    on='tag'
).drop_duplicates('id')[['quote','big_cate','tag','id','author']]

In [51]:
df_quote_1[['quote','big_cate','tag','id','author']].to_csv('/var/DSTeam/locnt2/personal/quotes-tagging/quote_v1.csv')

In [52]:
df_quote_1

,quote,big_cate,tag,id,author
0,"Only once in your life, I truly believe, you f...",relationships,love,6,Bob Marley
1,"As he read, I fell in love the way you fall as...",relationships,love,8,"John Green, The Fault in Our Stars"
2,"I love you without knowing how, or when, or fr...",relationships,love,11,"Pablo Neruda, 100 Love Sonnets"
3,Have you ever been in love? Horrible isn't it?...,relationships,love,13,"Neil Gaiman, The Kindly Ones"
4,There is never a time or place for true love. ...,relationships,love,16,"Sarah Dessen, The Truth About Forever"
...,...,...,...,...,...
66289,I have learned that not diamonds but divorce l...,relationships,best friend,458294,Zsa Zsa Gabor
66290,My mind has been both my best friend and my wo...,relationships,best friend,458348,Elyn Saks
66291,"If I stopped writing and being at my piano, I ...",relationships,best friend,458384,Jessi Colter
66292,I've learned that I am allowed to say what I l...,relationships,best friend,458416,Bobbi Brown


In [53]:
df_quote_1.tag.nunique()

77

In [54]:
df_quote_1.big_cate.nunique()

7

In [55]:
test_dict=df_quote_1.assign(quote_author_pair=lambda x:x.quote+'|author|'+x.author).groupby(['big_cate','tag']).agg(
    list_quote_author=('quote_author_pair',list)
).reset_index()

In [56]:
# test_dict=df_quote_1.assign(quote_author_pair=lambda x:x.quote+'|author|'+x.author).groupby(['big_cate','tag']).agg(
#     list_quote=('quote',list),
#     list_author=('author',list)
# ).reset_index()

In [57]:
dict_json={}
for i in range(len(test_dict)):
    row=test_dict.iloc[i]
    if row.big_cate not in dict_json:
        dict_json[row.big_cate]={}
    else:
        dict_json[row.big_cate][row.tag]=row.list_quote_author

In [61]:
num_quotes_per_partition=20
num_partition_per_tags=10
env='staging'
work_dir='/var/DSTeam/locnt2/personal/quotes-tagging'

In [62]:
import os
path = "pythonprog"
# Check whether the specified path exists or not
isExist = os.path.exists(path)
if not isExist:
   # Create a new directory because it does not exist
   os.makedirs(path)


In [63]:
def check_exist_next_partition(big_cate: str, tag: str, current_partition: int) -> bool:
    if (
        len(
            dict_json[big_cate][tag][
                (i + 1) * num_quotes_per_partition : (i + 2) * num_quotes_per_partition
            ]
        )
        > 0
    ):
        return True
    return False


def extract_quote_author_list(list_author_quote, sep="|author|"):
    # print(list_author_quote)
    try:
        return [
            {"item": x[0], "owner": x[1]}
            for x in list(
                map(
                    lambda y: y.split(sep),
                    list(filter(lambda z: isinstance(z, str), list_author_quote)),
                )
            )
        ]
    except:
        print(list_author_quote)

In [68]:
5//2

2

In [81]:
for big_cat in dict_json.keys():
    for tag in dict_json[big_cat].keys():
        num_partition_this_tag=len(dict_json[big_cat][tag])//num_quotes_per_partition
        num_partition_this_tag=num_partition_this_tag+int(num_quotes_per_partition*num_partition_this_tag<len(dict_json[big_cat][tag]))
        for i in range(min(num_partition_this_tag,num_partition_per_tags)):
            current_path="{}/data/{}/{}/{}".format(work_dir,big_cat,tag,i)
            if not os.path.exists(current_path):
                os.makedirs(current_path)
            with open(
                "{}/data/{}/{}/{}/data.json".format(work_dir,big_cat,tag,i), "w"
            ) as outfile:
                # prev=None if i==0 else "/data/{}/{}/{}/data.json".format(big_cat,tag,i-1)
                # next=None if i==num_partition_per_tags-1 or not check_exist_next_partition(big_cat,tag,i) else "/data/{}/{}/{}/data.json".format(big_cat,tag,i+1)
                contents=dict_json[big_cat][tag][i*num_quotes_per_partition:(i+1)*num_quotes_per_partition]
                if len(contents)>0:
                    json.dump(
                        {
                            'contents':extract_quote_author_list(contents),
                            'pagination':{
                                # 'next':next,
                                # 'prev':prev,
                                # 'current':"/data/{}/{}/{}/data.json".format(big_cat,tag,i)
                                "size":len(contents),
                                'total': min(num_partition_this_tag,num_partition_per_tags),
                                'current':i
                            }
                        },
                        outfile
                    )
                    # print('1')
                else:
                    shutil.rmtree(current_path)

In [29]:
modified_metadata={}

In [82]:
for big_cat in cate_roots.keys():
    modified_metadata[big_cat]=[]
    for tag in cate_roots[big_cat]:
        if os.path.exists('/var/DSTeam/locnt2/personal/quotes-tagging/data/{}/{}'.format(big_cat,tag)):
            obj={
                'url':'public/data/quotes/{}/{}/0/data.json'.format(big_cat,tag),
                'tag':tag
            } 
            modified_metadata[big_cat].append(obj)
        

In [83]:
with open(
    "/var/DSTeam/locnt2/personal/quotes-tagging/data/category.json", "w"
) as outfile:
    json.dump(modified_metadata, outfile)

In [34]:
df_quote_remain=pd.merge(
    df,
    df_quote_1[['id']],
    on='id',
    how='outer',indicator=True
).loc[lambda x:x._merge=='left_only'].drop('_merge',axis=1)

In [35]:
df_quote_remain.to_csv('/var/DSTeam/locnt2/personal/quotes-tagging/remain_quote.csv',index=False)

## Mapping from tags to predefined cate

In [36]:
all_tags=df.tags.str.lower().str.split(',').explode('tags').drop_duplicates().tolist()

In [37]:
cate_pd_total

,big_cate,tag
0,hard times,heartbroken
1,hard times,breakup
2,hard times,overcoming fear
3,hard times,be strong
4,hard times,overthinking
...,...,...
9,health & fitness,no excuses
10,health & fitness,body positivity
11,health & fitness,training
12,health & fitness,running


In [38]:
df_quote_remain

,quote,author,tags,id
0,"I'm selfish, impatient and a little insecure. ...",Marilyn Monroe,"attributed-no-source, best, life, love, mistak...",0
1,You've gotta dance like there's nobody watchin...,William W. Purkey,"dance, heaven, hurt, inspirational, life, love...",1
2,You know you're in love when you can't fall as...,Dr. Seuss,"attributed-no-source, dreams, love, reality, s...",2
3,A friend is someone who knows all about you an...,Elbert Hubbard,"friend, friendship, knowledge, love",3
4,Darkness cannot drive out darkness: only light...,"Martin Luther King Jr., A Testament of Hope: T...","darkness, drive-out, hate, inspirational, ligh...",4
...,...,...,...,...
499702,The future isn't just a place you'll go. It's ...,Nancy Duarte,"You, Place, Will",499702
499703,The Christian of the future will be a mystic o...,Karl Rahner,"Christian, Will, Exist",499703
499704,I do believe the most important thing I can do...,John C. Stennis,"Past, Believe, Help",499704
499705,I'd say I'm a bit antimadridista although I do...,Isco,"Team, Humility, Know",499705


In [40]:
from gensim.test.utils import common_texts
from gensim.models import Word2Vec
import gensim.downloader


In [129]:
print(list(gensim.downloader.info()['models'].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [41]:
w2v_vectors = gensim.downloader.load('word2vec-google-news-300')


In [42]:
sims = w2v_vectors.most_similar_to_given('apple',['shit','banana','laptop'])  

In [39]:
w2v_vectors.distance('apple','red apple')

NameError: name 'w2v_vectors' is not defined

In [171]:
sims

'banana'